<a href="https://colab.research.google.com/github/AdityaBisht099/Collab-Board/blob/main/Anaverse_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ana-verse-2-0-n/train.parquet
/kaggle/input/ana-verse-2-0-n/sample_submission.parquet
/kaggle/input/ana-verse-2-0-n/test.parquet


In [ ]:
train_df = pd.read_parquet('/kaggle/input/ana-verse-2-0-n/train.parquet')
test_df  = pd.read_parquet('/kaggle/input/ana-verse-2-0-n/test.parquet')
sample_sub = pd.read_parquet('/kaggle/input/ana-verse-2-0-n/sample_submission.parquet')

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
print("Sample submission shape:", sample_sub.shape)

train_df.head()

Train shape: (1639424, 7)
Test shape: (409856, 7)
Sample submission shape: (50, 2)


,Date,X1,X2,X3,X4,X5,target
0,2020-12-16,1.518921,5.463154,1.0,2.718282,2.890372,0
1,2020-12-16,1.546509,5.458010,1.0,2.718282,2.833213,1
2,2020-12-16,1.645427,5.456560,1.0,7.389056,2.890372,1
3,2020-12-16,1.652022,5.458479,1.0,2.718282,2.890372,1
4,2020-12-16,1.695538,5.466709,1.0,2.718282,2.890372,0


In [ ]:
X = train_df.drop(columns=['target'])
y = train_df['target']

print("Features shape:", X.shape)
print("Target shape:", y.shape)

y.value_counts(normalize=True)

Features shape: (1639424, 6)
Target shape: (1639424,)


target
0    0.991437
1    0.008563
Name: proportion, dtype: float64

In [ ]:
missing_values = X.isnull().sum()

missing_values[missing_values > 0]

Series([], dtype: int64)

In [ ]:
X['Date'] = pd.to_datetime(X['Date'])

X['year'] = X['Date'].dt.year
X['month'] = X['Date'].dt.month
X['day'] = X['Date'].dt.day

X = X.drop(columns=['Date'])

X.head()

,X1,X2,X3,X4,X5,year,month,day
0,1.518921,5.463154,1.0,2.718282,2.890372,2020,12,16
1,1.546509,5.458010,1.0,2.718282,2.833213,2020,12,16
2,1.645427,5.456560,1.0,7.389056,2.890372,2020,12,16
3,1.652022,5.458479,1.0,2.718282,2.890372,2020,12,16
4,1.695538,5.466709,1.0,2.718282,2.890372,2020,12,16


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.1,
    random_state=42,
    stratify=y
)

print("Train size:", X_train.shape)
print("Validation size:", X_val.shape)


Train size: (1311539, 8)
Validation size: (327885, 8)


In [ ]:
y_train = y_train.astype(int)
y_val = y_val.astype(int)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, accuracy_score

log_reg = LogisticRegression(
    max_iter=1000,
    class_weight='balanced',
)

log_reg.fit(X_train_scaled, y_train)

y_val_pred = log_reg.predict(X_val_scaled)

print("Accuracy:", accuracy_score(y_val, y_val_pred))
print("F1 Score:", f1_score(y_val, y_val_pred))
print("\nClassification Report:\n")
print(classification_report(y_val, y_val_pred))


NameError: name 'X_train_scaled' is not defined

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    min_samples_split=5,
    min_samples_leaf=2,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)

y_val_pred_rf = rf_model.predict(X_val)

print("Accuracy:", accuracy_score(y_val, y_val_pred_rf))
print("F1 Score:", f1_score(y_val, y_val_pred_rf))
print("\nClassification Report:\n")
print(classification_report(y_val, y_val_pred_rf))


In [ ]:
y_val_probs = rf_model.predict_proba(X_val)[:, 1]


In [ ]:
import numpy as np
from sklearn.metrics import f1_score

thresholds = np.arange(0.1, 0.85, 0.04)
best_f1 = 0
best_threshold = 0.5

for t in thresholds:
    y_pred_thresh = (y_val_probs >= t).astype(int)
    f1 = f1_score(y_val, y_pred_thresh)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = t

print("Best Threshold:", best_threshold)
print("Best F1 Score:", best_f1)


In [ ]:
rf_model.fit(X, y)

In [ ]:
X_test = test_df.copy()

X_test['Date'] = pd.to_datetime(X_test['Date'])
X_test['year'] = X_test['Date'].dt.year
X_test['month'] = X_test['Date'].dt.month
X_test['day'] = X_test['Date'].dt.day

X_test = X_test.drop(columns=['Date', 'ID'])

X_test.head()


In [ ]:
test_probs = rf_model.predict_proba(X_test)[:, 1]

test_preds = (test_probs >= best_threshold).astype(int)

import numpy as np
np.unique(test_preds, return_counts=True)


In [ ]:
submission = pd.DataFrame({
    'ID': test_df['ID'],
    'target': test_preds
})

submission.head(), submission.shape


In [ ]:
submission.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
test_df['Date'] = pd.to_datetime(test_df['Date'])

test_df['year'] = test_df['Date'].dt.year
test_df['month'] = test_df['Date'].dt.month
test_df['day'] = test_df['Date'].dt.day


In [ ]:
test_X = test_df.drop(columns=['ID', 'Date'], errors='ignore')

test_preds = rf_model.predict(test_X)

submission = pd.DataFrame({
    'ID': test_df['ID'],
    'target': test_preds
})

submission.to_csv('/kaggle/working/submission.csv', index=False)

submission.head()
